In [162]:
## Used for vectorizing the raw data (run it once on train and once on test) :
## Pivoting it from the initial feature_name:feature_value form to a vector
## scalar_feature_to_dummies - Translating categoric variables into N-1 dummy variables
## timeseries_feature_slope_reduced - mean, std for time series variables (have multiple measurements in different times)
## timeseries_feature_last_value - take last value in time series
## Filling empty values with means - NOTE that these have to be the train data means

In [163]:
import pandas as pd
import numpy as np

In [164]:
df = pd.read_csv('../train_data.csv', sep = '|', error_bad_lines=False, index_col=False, dtype='unicode')
df.describe()

,SubjectID,form_name,feature_name,feature_value,feature_unit,feature_delta
count,1138647,1138647,1138647,1138635,727028,1136734
unique,1777,12,1337,36409,48,12952
top,25028,Lab Test,Q2_Salivation,4.0,mmol/L,0.0
freq,1069,615019,18879,83967,169840,74413


In [165]:
interesting = df[(df.form_name == 'Demographic') | (df.form_name == 'Vitals')]
print interesting['feature_name'].unique()
func_per_feature = {}
vectorized = pd.DataFrame(index=df['SubjectID'].unique())
print vectorized.shape

['Gender' 'Age' 'Race' 'bp_diastolic' 'bp_systolic' 'pulse'
 'respiratory_rate' 'temperature' 'weight' 'height' 'BMI']
(1777, 0)


In [166]:
def scalar_feature_to_dummies(df, feature_name):
    my_slice = df[df.feature_name == feature_name]
    my_slice_pivot = pd.pivot_table(my_slice, values = ['feature_value'], index = ['SubjectID'], 
                                columns = ['feature_name'], aggfunc = lambda x:x)
    dum = pd.get_dummies(my_slice_pivot['feature_value'][feature_name])
    return dum

for feature_name in ['Gender', 'Race']:
    func_per_feature[feature_name] = scalar_feature_to_dummies
    vectorized = pd.merge(vectorized, func_per_feature[feature_name](df, feature_name), 
                          right_index=True, left_index=True)  

vectorized.head()


,F,M,Asian,Black,Hispanic,Other,Unknown,White
100256,0,1,0,0,0,0,0,1
100626,0,1,0,0,0,0,0,1
100806,0,1,0,0,0,0,0,1
100981,0,1,0,0,0,0,0,1
101154,0,1,0,0,0,0,0,1


In [167]:
### Calculating slope - the diffs between each measurement and the first measurement (0 day) 
def calc_slope(row) :
    time_delta =  (float(row['feature_delta_int_y']) - float(row['feature_delta_int_x']))
    return (row['feature_value_float_y'] - row['feature_value_float_x'])/time_delta

def timeseries_feature_to_slope(df, feature_name):
    my_slice = df[df.feature_name == feature_name]
    # There were duplicate measurements of timeseries features with the same feature_delta :(
    my_slice = my_slice.drop_duplicates(subset = ['SubjectID', 'feature_delta'], take_last=True)
    my_slice.loc[:, 'feature_value_float'] = my_slice['feature_value'].astype(float)
    my_slice.loc[:, 'feature_delta_int'] = my_slice['feature_delta'].astype(int)
    my_slice_other_visits = my_slice[(my_slice.feature_delta_int > 0) & (my_slice.feature_delta_int < 92)]
    my_slice_first_visit = my_slice[my_slice.feature_delta_int == 0]
    my_slice_j = pd.merge(my_slice_first_visit, my_slice_other_visits, on=['SubjectID','feature_name']) 
    my_slice_j.loc[:, 'feature_value_slope'] = my_slice_j.apply(calc_slope, axis=1)
    return my_slice_j

def timeseries_feature_slope_reduced(df, feature_name):
    res = pd.DataFrame(index=df['SubjectID'].unique())
    for func in ['mean', 'std']:
        slope_series = timeseries_feature_to_slope(df, feature_name)
        slope_pivot = pd.pivot_table(slope_series, values = ['feature_value_slope'], index = ['SubjectID'], 
                                     columns = ['feature_name'], aggfunc = func)
        slope_pivot = slope_pivot['feature_value_slope']
        slope_pivot.columns = [feature_name + "_" + func]
        res = pd.merge(res, slope_pivot, right_index=True, left_index=True)          
    return res

for feature_name in ['bp_diastolic', 'bp_systolic', 'pulse', 'respiratory_rate', 'temperature', 'weight']:
    func_per_feature[feature_name] = timeseries_feature_slope_reduced
    vectorized = pd.merge(vectorized, func_per_feature[feature_name](df, feature_name), 
                          right_index=True, left_index=True)  

vectorized.head()

,F,M,Asian,Black,Hispanic,Other,Unknown,White,bp_diastolic_mean,bp_diastolic_std,bp_systolic_mean,bp_systolic_std,pulse_mean,pulse_std,respiratory_rate_mean,respiratory_rate_std,temperature_mean,temperature_std,weight_mean,weight_std
100256,0,1,0,0,0,0,0,1,-0.516411,0.492568,-0.340148,0.528434,0.440030,0.379330,0.009804,0.019608,0.037261,0.055500,-0.141928,0.113586
101645,0,1,0,0,0,0,0,1,-0.223817,0.189501,-0.258505,0.151541,0.008051,0.187971,-0.215146,0.237859,0.011515,0.021696,0.020049,0.019091
103126,0,1,0,0,0,0,1,0,-0.148810,0.260956,-0.285714,0.336718,0.172619,0.159941,0.000000,0.000000,-0.000595,0.005893,-0.010714,0.045457
104403,0,1,0,0,0,0,0,1,0.045777,0.236461,0.204257,0.639180,-0.168992,0.521850,0.062857,0.091250,-0.020451,0.039782,0.004854,0.015666
104703,1,0,0,0,0,0,0,1,-0.319444,0.176777,0.347222,0.451763,-0.402778,0.687465,0.000000,0.000000,0.013194,0.012767,0.035417,0.028481


In [168]:
def timeseries_feature_last_value(df, feature_name):
    my_slice = df[df.feature_name == feature_name]
    ret = my_slice.groupby('SubjectID').last().loc[:, ['feature_value']]
    ret.columns = [feature_name + "_last"]
    return ret

for feature_name in ['BMI', 'height']:
    func_per_feature[feature_name] = timeseries_feature_last_value
    vectorized = pd.merge(vectorized, func_per_feature[feature_name](df, feature_name), 
                          right_index=True, left_index=True)  

vectorized.head()

,F,M,Asian,Black,Hispanic,Other,Unknown,White,bp_diastolic_mean,bp_diastolic_std,...,pulse_mean,pulse_std,respiratory_rate_mean,respiratory_rate_std,temperature_mean,temperature_std,weight_mean,weight_std,BMI_last,height_last
103126,0,1,0,0,0,0,1,0,-0.148810,0.260956,...,0.172619,0.159941,0.000000,0.000000,-0.000595,0.005893,-0.010714,0.045457,0.0021820987654321,180.0
104703,1,0,0,0,0,0,0,1,-0.319444,0.176777,...,-0.402778,0.687465,0.000000,0.000000,0.013194,0.012767,0.035417,0.028481,0.0027053202479338802,176.0
107828,1,0,0,0,0,0,0,1,-0.206522,0.415041,...,0.304348,0.512396,0.514493,0.686611,0.092391,0.128611,0.047826,0.026645,0.0023384353741496603,168.0
111580,0,1,0,0,0,0,0,1,0.275862,NaN,...,0.068966,NaN,0.000000,NaN,-0.017241,NaN,-0.017241,NaN,0.00317194798636536,178.0
112203,1,0,0,0,0,0,0,1,0.064286,0.111117,...,-0.442857,0.585888,0.000000,0.000000,0.026429,0.023234,-0.024286,0.046467,0.0025098855359001,155.0


In [169]:
## Filling empty values with means - NOTE that these have to be the train data means
train_data_means = vectorized.mean()
vectorized = vectorized.fillna(train_data_means)
vectorized.head()

,F,M,Asian,Black,Hispanic,Other,Unknown,White,bp_diastolic_mean,bp_diastolic_std,...,pulse_mean,pulse_std,respiratory_rate_mean,respiratory_rate_std,temperature_mean,temperature_std,weight_mean,weight_std,BMI_last,height_last
103126,0,1,0,0,0,0,1,0,-0.148810,0.260956,...,0.172619,0.159941,0.000000,0.000000,-0.000595,0.005893,-0.010714,0.045457,0.0021820987654321,180.0
104703,1,0,0,0,0,0,0,1,-0.319444,0.176777,...,-0.402778,0.687465,0.000000,0.000000,0.013194,0.012767,0.035417,0.028481,0.0027053202479338802,176.0
107828,1,0,0,0,0,0,0,1,-0.206522,0.415041,...,0.304348,0.512396,0.514493,0.686611,0.092391,0.128611,0.047826,0.026645,0.0023384353741496603,168.0
111580,0,1,0,0,0,0,0,1,0.275862,0.512172,...,0.068966,0.578307,0.000000,0.155443,-0.017241,0.031825,-0.017241,0.053787,0.00317194798636536,178.0
112203,1,0,0,0,0,0,0,1,0.064286,0.111117,...,-0.442857,0.585888,0.000000,0.000000,0.026429,0.023234,-0.024286,0.046467,0.0025098855359001,155.0


In [170]:
for t in ["train", "test"]:
    df = pd.read_csv('../' + t + '_data.csv', sep = '|', error_bad_lines=False, index_col=False, dtype='unicode')
    vectorized = pd.DataFrame(index=df['SubjectID'].unique())
    for feature_name, func in func_per_feature.iteritems():
        vectorized = pd.merge(vectorized, func_per_feature[feature_name](df, feature_name), 
                      right_index=True, left_index=True)  
    final_data = vectorized.fillna(train_data_means)
    final_data.to_csv('../' + t + '_data_vectorized.csv' ,sep='|')

In [171]:
func_per_feature

{'BMI': <function __main__.timeseries_feature_last_value>,
 'Gender': <function __main__.scalar_feature_to_dummies>,
 'Race': <function __main__.scalar_feature_to_dummies>,
 'bp_diastolic': <function __main__.timeseries_feature_slope_reduced>,
 'bp_systolic': <function __main__.timeseries_feature_slope_reduced>,
 'height': <function __main__.timeseries_feature_last_value>,
 'pulse': <function __main__.timeseries_feature_slope_reduced>,
 'respiratory_rate': <function __main__.timeseries_feature_slope_reduced>,
 'temperature': <function __main__.timeseries_feature_slope_reduced>,
 'weight': <function __main__.timeseries_feature_slope_reduced>}